In [9]:
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.simulations.epidemic import Epidemic
from autoemulate.experimental.calibration.history_matching import HistoryMatchingWorkflow
from autoemulate.history_matching_dashboard import HistoryMatchingDashboard


# History Matching workflow

The `HistoryMatchingWorkflow` implements an iterative sample-predict-refit workflow:
- sample `n_test_samples` to test from the not ruled out yet (NROY) space
- use emulator to filter out implausible samples and update the NROY space
- run `n_simulations` predictions for the sampled parameters using the simulator
- refit the emulator using the simulated data


## 1. Simulate data and train an emulator

In [10]:
simulator = Epidemic()
x = simulator.sample_inputs(1000)
y = simulator.forward_batch(x)

Running simulations: 100%|██████████| 1.00k/1.00k [00:00<00:00, 1.45ksample/s]


In [11]:
ae = AutoEmulate(x, y, models=[GaussianProcessExact])

Comparing models: 100%|██████████| 1.00/1.00 [00:20<00:00, 20.5s/model]


In [12]:
ae.summarise()

,model_name,x_transforms,y_transforms,rmse_score,r2_score
0,GaussianProcessExact,[StandardizeTransform()],[StandardizeTransform()],0.000452,0.991869


In [13]:
model = ae.best_result().model

## 2. Calibrate

Instantiate the `HistoryMatchingWorkflow` object with an observed mean and variance for each simulator output as well as the simulator and trained emulator model.

In [14]:
observations = {"infection_rate": (0.3, 0.05)}

In [15]:
hmw = HistoryMatchingWorkflow(
    simulator=simulator,
    emulator=model,
    observations=observations,
    threshold=3.0,
    train_x=x,
    train_y=y
)


The object maintains and updates the internal state each time `run()` is called so the full workflow can be run over a number of iterations.

In [16]:
test_parameters, impl_scores = hmw.run(n_simulations=20, n_test_samples=100)

Running simulations: 100%|██████████| 20.0/20.0 [00:00<00:00, 1.21ksample/s]


## 3. Visualise results

In [17]:
dashboard = HistoryMatchingDashboard(
    samples=test_parameters,
    impl_scores=impl_scores,
    param_names=simulator.param_names,  
    output_names=simulator.output_names, 
)

In [18]:
dashboard.display()

HTML(value='<h2>History Matching Dashboard</h2>')